---
# Welcome to CE880
### This is your week-10 : Problem notebook

For this problem set, we'll be using the Jupyter notebook and please upload this notebook to [Google Colab](https://colab.research.google.com/). 

Paris Housing is a set of data created from imaginary data of house prices in an urban environment. 
https://github.com/sagihaider/CE880_2021/blob/main/Data/ParisHousingClass.csv 

All attributes are numeric variables and they are listed bellow:
* squareMeters
* numberOfRooms
* hasYard
* hasPool
* floors - number of floors
* cityCode - zip code
* cityPartRange - the higher the range, the more exclusive the neighbourhood is
* numPrevOwners - number of prevoious owners
* made - year
* isNewBuilt
* hasStormProtector
* basement - basement square meters
* attic - attic square meteres
* garage - garage size
* hasStorageRoom
* hasGuestRoom - number of guest rooms
* price - price of a house
* category - Luxury or Basic

In [1]:
import numpy as np 
import pandas as pd 
url = 'https://raw.githubusercontent.com/sagihaider/CE880_2021/main/Data/ParisHousingClass.csv'
trainData = pd.read_csv(url,index_col=0)
print(trainData.head())

              numberOfRooms  hasYard  hasPool  floors  cityCode  \
squareMeters                                                      
75523                     3        0        1      63      9373   
80771                    39        1        1      98     39381   
55712                    58        0        1      19     34457   
32316                    47        0        0       6     27939   
70429                    19        1        1      90     38045   

              cityPartRange  numPrevOwners  made  isNewBuilt  \
squareMeters                                                   
75523                     3              8  2005           0   
80771                     8              6  2015           1   
55712                     6              8  2021           0   
32316                    10              4  2012           0   
70429                     3              7  1990           1   

              hasStormProtector  basement  attic  garage  hasStorageRoom  \
squar

In [4]:
from sklearn.preprocessing import LabelEncoder

def label_encoded(feat):
    le=LabelEncoder()
    le.fit(feat)
    print(feat.name,le.classes_)
    return le.transform(feat)
trainData['category']=label_encoded(trainData['category'])

category ['Basic' 'Luxury']


In [5]:
from sklearn.model_selection import train_test_split,GridSearchCV

y=trainData['category']
x=trainData.drop('category',axis=1)
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

---
## Question 1: 

#### Classification
Use the dataset, which I have spilted into training and test sets (xtrain, xtest, ytrain, ytest), please train the machine learning model to get the test accuracy of 100%. You are free to use any classication model such as Decision Tree, Random Forest, KNN, SVM..etc. 

Hint: Please use grid search to find a best model using different parameters. 

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score as accuracy
def my_model(xtrain,xtest,ytrain,ytest):
    """Write a function to train a machine learning model to get the test accuracy of 100%. 
    You are free to use any classication model such as Decision Tree, 
    Random Forest, KNN, SVM..etc. """
    # YOUR CODE HERE
    xgb_clf = xgb.XGBClassifier()
    parameters = {'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
                 }
    grid_GBR = GridSearchCV(estimator=xgb_clf, param_grid = parameters, cv = 2, n_jobs=-1)
    grid_GBR.fit(xtrain, ytrain)
    print(" Results from Grid Search " )
    print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
    print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
    print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)
    grid_values = grid_GBR.best_params_
    colsample_bytree_values = grid_values['colsample_bytree']
    gamma_values = grid_values['gamma']
    max_depth_values = grid_values['max_depth']
    min_child_weight_values = grid_values['min_child_weight']
    subsample_values = grid_values['subsample']
    xgb_clf = xgb.XGBClassifier(colsample_bytree = colsample_bytree_values, gamma = gamma_values, max_depth = max_depth_values, min_child_weight = min_child_weight_values, subsample = min_child_weight_values)
    xgb_clf = xgb_clf.fit(xtrain, ytrain)
    y_predict_xgb = xgb_clf.predict(xtest)
    # Print results: 
    print(confusion_matrix(ytest, y_predict_xgb))
    print(classification_report(ytest, y_predict_xgb))
    return accuracy(y_predict_xgb,ytest)*100
    raise NotImplementedError()

In [15]:
# Check you solution by running this cell
import math
assert math.isclose(my_model(xtrain,xtest,ytrain,ytest), 100.0, rel_tol = 0.05)

 Results from Grid Search 

 The best estimator across ALL searched params:
 XGBClassifier(colsample_bytree=0.6, gamma=0.5, subsample=0.6)

 The best score across ALL searched params:
 1.0

 The best parameters across ALL searched params:
 {'colsample_bytree': 0.6, 'gamma': 0.5, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.6}
[[1744    0]
 [   0  256]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1744
           1       1.00      1.00      1.00       256

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

